<p style="text-align:center">
    <a href="https://www.ptit.edu.vn" target="_blank">
        <img src="https://i.imgur.com/XBSxCH1.png" width="400" alt="PTIT Logo">
    </a>
</p>


<center>

# Practice lab: Cardiovascular Prediction

</center>


Reference: https://www.sciencedirect.com/science/article/abs/pii/S0957417420302323



### MỤC TIÊU VÀ LÝ DO NGHIÊN CỨU

- Mục tiêu nghiên cứu: Nghiên cứu này nhằm phát hiện một mạng nơ-ron tích chập để dự đoán **bệnh tim mạch vành (CHD)** dựa trên dữ liệu sẵn có của **NHANES**, nghiên cứu tập trung vào cải thiện độ chính xác trong việc chẩn đoán **CHD** 
- Lý do: **Bệnh tim mạch vành** là một trong những nguyên nhân chính gây tử vong trên toàn thế giới, việc chẩn đoán sớm có thể giúp giảm nguy cơ và phòng ngừa các cơn đau tim. Tuy nhiên việc sử dụng các mô hình học máy, học sâu truyền thống thường gặp khó khăn khi áp dụng bộ dữ liệu này

### PHƯƠNG PHÁP NGHIÊN CỨU

- Bộ dữ liệu bao gồm thông tin lâm sàng và nhân khẩu học của 37079 cá nhân (1300 ca CHD và 35779 ca Non-CHD)
- Sử dụng các mô hình học máy như Random Forest, SVM và học sâu CNN để thử với bộ dữ liệu trên

### THƯ VIỆN

In [1]:
import pandas as pd
import numpy as np


### ĐỌC DỮ LIỆU TỪ DATASET

In [2]:
file_path = r'C:\Users\baoho\Desktop\PTIT-CNN-Prediction-Cardiovascular\CardiacPrediction.xlsx'
df = pd.read_excel(file_path)

### KHAI PHÁ DỮ LIỆU

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37079 entries, 0 to 37078
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SEQN                         37079 non-null  int64  
 1   Gender                       37079 non-null  int64  
 2   Age                          37079 non-null  int64  
 3   Annual-Family-Income         37079 non-null  int64  
 4   Ratio-Family-Income-Poverty  37079 non-null  float64
 5   X60-sec-pulse                37079 non-null  int64  
 6   Systolic                     37079 non-null  int64  
 7   Diastolic                    37079 non-null  int64  
 8   Weight                       37079 non-null  float64
 9   Height                       37079 non-null  float64
 10  Body-Mass-Index              37079 non-null  float64
 11  White-Blood-Cells            37079 non-null  float64
 12  Lymphocyte                   37079 non-null  float64
 13  Monocyte        

In [4]:
df.isnull().sum()

SEQN                           0
Gender                         0
Age                            0
Annual-Family-Income           0
Ratio-Family-Income-Poverty    0
X60-sec-pulse                  0
Systolic                       0
Diastolic                      0
Weight                         0
Height                         0
Body-Mass-Index                0
White-Blood-Cells              0
Lymphocyte                     0
Monocyte                       0
Eosinophils                    0
Basophils                      0
Red-Blood-Cells                0
Hemoglobin                     0
Mean-Cell-Vol                  0
Mean-Cell-Hgb-Conc.            0
Mean-cell-Hemoglobin           0
Platelet-count                 0
Mean-Platelet-Vol              0
Segmented-Neutrophils          0
Hematocrit                     0
Red-Cell-Distribution-Width    0
Albumin                        0
ALP                            0
AST                            0
ALT                            0
Cholestero

❌ Không có giá trị NULL trong bộ dữ liệu 

In [5]:
# I want to see std, min25%, min50%, min75%, max of each column
df.describe()

,SEQN,Gender,Age,Annual-Family-Income,Ratio-Family-Income-Poverty,X60-sec-pulse,Systolic,Diastolic,Weight,Height,...,Total-Cholesterol,HDL,Glycohemoglobin,Vigorous-work,Moderate-work,Health-Insurance,Diabetes,Blood-Rel-Diabetes,Blood-Rel-Stroke,CoronaryHeartDisease
count,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,...,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000
mean,48901.041236,1.513282,48.943661,7.358208,2.559026,72.579250,124.090078,69.919253,80.988276,167.389601,...,5.081713,1.370344,5.676496,1.783840,1.598856,1.218587,1.907333,1.549502,1.796165,0.040670
std,26753.636441,0.499830,18.010440,3.994083,1.624789,12.242108,19.254741,13.575804,20.678734,10.122908,...,1.072682,0.415985,1.050223,0.448324,0.511199,0.461102,0.349674,0.497550,0.402853,0.197527
min,2.000000,1.000000,20.000000,1.000000,0.000000,32.000000,0.000000,0.000000,32.300000,129.700000,...,1.530000,0.160000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,26120.500000,1.000000,33.000000,4.000000,1.140000,64.000000,111.000000,62.000000,66.500000,160.000000,...,4.320000,1.070000,5.200000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000
50%,50065.000000,2.000000,48.000000,7.000000,2.180000,72.000000,121.000000,70.000000,78.200000,167.100000,...,5.020000,1.290000,5.400000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
75%,71173.500000,2.000000,63.000000,10.000000,4.130000,80.000000,134.000000,78.000000,92.100000,174.600000,...,5.740000,1.600000,5.800000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
max,93702.000000,2.000000,85.000000,15.000000,5.000000,224.000000,270.000000,132.000000,371.000000,204.500000,...,14.090000,5.840000,18.800000,3.000000,3.000000,9.000000,3.000000,2.000000,2.000000,1.000000


### GIẢI THÍCH BỘ DỮ LIỆU

1. Nhân khẩu học (Demographic)
- Age: Tuổi của người tham gia khảo sát
- Gender: Giới tính (1 - Nam, 2 - Nữ)
2. Biến số kiểm tra (Examination Variables)
- Height: Chiều cao (cm)
- Weight: Cân nặng (kg)
- Huyết áp (Systolic/Diastolic): Huyết áp tâm thu và huyết áp tâm trương là 2 chỉ số quan trọng để chẩn đoán bệnh nhân mắc bệnh tim mạch
- BMI: Chỉ số khối cơ thể, những người có chỉ số BMI cao thường dễ mắc các bệnh tim mạch hơn bình thường
3. Biến số thí nghiệm (Laboratory Variables)
- Mức độ Choresterol: Ba gồm các chỉ số Choresterol toàn phần như LDL (Low-Density Lipprotein) và HDL (High-Density Lipprotein). Choresterol cao là một trong những nguyên nhân hàng đầu gây ra bệnh tim mạch
- Glucose: Mức đường trong máu, liên quan mật thiết đến bệnh tiểu đường, một yếu tố nguy hiểm đối với người bệnh tim
- Triglycerides: Mức chất béo trong máu
- Serum Albumin, Lactate Dehydrogenase (LDH): Chỉ số sinh hóa máu liên quan đến sức khỏe tổng quát và nguy cơ mắc bệnh tim mạch
4. Biến số khảo sát (Questionare Variables)
- Tình trạng hút thuốc lá
- Tình trạng bệnh tiểu đường
- Physical Activity (Hoạt động thể chất): Hoạt động thể chất mạnh, vừa phải có liên quan đến bệnh tim mạch
5. Biến số độc lập
- CoronaryHeartDisease: Biến nhị phân biểu thị việc người tham gia khảo sát có mắc bệnh tim mạch vành không (1 - Có, 2 - Không)

In [7]:
# I want to know how many 0 and how many 1 in column CHD
df['CoronaryHeartDisease'].value_counts()

CoronaryHeartDisease
0    35571
1     1508
Name: count, dtype: int64

📶  Có thể thấy sự mất cân bằng dữ liệu rất lớn trên tập dữ liệu này với tỉ lệ 1:35 (Người mắc bệnh : Người không mắc bệnh)